In [6]:
import torch
print("cuda is available:",torch.cuda.is_available())
if torch.cuda.is_available():
    print("Device name: ",torch.cuda.get_device_name(0))

cuda is available: True
Device name:  NVIDIA GeForce RTX 4070 Laptop GPU


In [7]:
import os
import glob
from diffusers import StableDiffusionUpscalePipeline
import torch
from PIL import Image

def upscale_single_image(input_folder, output_folder):
    if not torch.cuda.is_available():
        print("GPU를 찾을 수 없습니다. 이 모델은 GPU 없이는 매우 느립니다.")
        return

    print("Stable Diffusion 업스케일링 모델을 로드 중...")
    try:
        model_id = "stabilityai/stable-diffusion-x4-upscaler"
        pipeline = StableDiffusionUpscalePipeline.from_pretrained(model_id, torch_dtype=torch.float16)
        pipeline = pipeline.to("cuda")
    except Exception as e:
        print(f"모델 로드 실패: {e}")
        return
    print("모델 로드 완료.")

    os.makedirs(output_folder, exist_ok=True)

    image_extensions = ['*.jpg', '*.jpeg', '*.png']
    image_path = None
    for ext in image_extensions:
        files = glob.glob(os.path.join(input_folder, ext))
        if files:
            image_path = files[0]
            break

    if not image_path:
        print(f"입력 폴더({input_folder})에서 이미지 파일을 찾을 수 없습니다.")
        return

    print(f"테스트 이미지 처리 중: {os.path.basename(image_path)}")
    
    # -------- 여기를 수정합니다 --------
    # 업스케일링할 이미지에 대한 텍스트 프롬프트
    # 이미지 내용에 맞게 구체적으로 작성할수록 좋습니다.
    prompt = "high-quality photo of a crumpled plastic wrapper on a concrete floor, high-resolution, sharp focus, detailed"
    # ---------------------------------
    
    try:
        low_res_img = Image.open(image_path).convert("RGB")
        low_res_img_resized = low_res_img.resize((256, 256))
        
        # 수정된 부분: prompt 인자 추가
        upscaled_image = pipeline(prompt=prompt, image=low_res_img_resized).images[0]

        final_image = upscaled_image.resize((1024, 1024), Image.LANCZOS)

        file_name = f"upscaled_{os.path.basename(image_path)}"
        output_path = os.path.join(output_folder, file_name)
        
        final_image.save(output_path)
        
        print(f"-> 테스트 이미지 저장 완료: {output_path}")

    except Exception as e:
        print(f"오류 발생: {os.path.basename(image_path)} - {e}")

if __name__ == "__main__":
    input_folder = "C:/project_aug/data/Dataset_project4/비닐"
    output_folder = "C:/project_aug/upscaled_images_sd"
    upscale_single_image(input_folder, output_folder)

Stable Diffusion 업스케일링 모델을 로드 중...


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

모델 로드 완료.
테스트 이미지 처리 중: 100000@4_06001_220808_P1_T1.jpg


  0%|          | 0/75 [00:00<?, ?it/s]

-> 테스트 이미지 저장 완료: C:/project_aug/upscaled_images_sd\upscaled_100000@4_06001_220808_P1_T1.jpg
